### read me text for requirements
<ul>
    <li> pandas== 1.0.5
    <li> numpy== 1.18.5
    <li> nltk== 3.2.5
    <li> gast== 0.2.2
    <li> tensorflow== 1.15.2
    <li> bert-tensorflow == 1.0.1
    <li> tensorflow-hub == 
    <li> tqdm==
    <li> ipywidgets==
        
</ul>

In [1]:
import pandas as pd 
import numpy as np
import nltk
from tqdm import tqdm_notebook 
from ipywidgets import IntProgress

import tensorflow as tf
import tensorflow_hub as hub
from bert.tokenization import FullTokenizer
from tensorflow.keras import backend as K
sess = tf.Session()


# input job description


In [2]:
# as if input was given on website 
jd = input('Paste job description here');

Paste job description hereC3.ai is a leading enterprise AI software provider for accelerating digital transformation. The comprehensive and proven C3 AI Suite uses a model-driven abstraction layer to enable organizations to develop, deploy, and operate enterprise scale AI applications 40x to 100x faster than alternative approaches. www.c3.ai  As a Data Scientist, you will participate in the definition of new analytics capabilities able to provide our customers with the information they need to make proper decisions to support our customers in operating the internet of things (IoT). In addition, you will help find the appropriate machine learning / data mining algorithms to answer these questions. Finally, you will be responsible for implementing this into the product and making it available to our customers.  Qualified candidates will have an in-depth knowledge of most common machine learning techniques and their application. You will also understand the limitations of these algorithms

# Tokenize an Tag POS

In [3]:
# input must be tokenized and pos tagging is done with vamnilla nltk pos tag
tokens = nltk.word_tokenize(jd.lower())
tagged = nltk.pos_tag(tokens)
tagged

[('c3.ai', 'NN'),
 ('is', 'VBZ'),
 ('a', 'DT'),
 ('leading', 'VBG'),
 ('enterprise', 'NN'),
 ('ai', 'NN'),
 ('software', 'NN'),
 ('provider', 'NN'),
 ('for', 'IN'),
 ('accelerating', 'VBG'),
 ('digital', 'JJ'),
 ('transformation', 'NN'),
 ('.', '.'),
 ('the', 'DT'),
 ('comprehensive', 'JJ'),
 ('and', 'CC'),
 ('proven', 'JJ'),
 ('c3', 'NN'),
 ('ai', 'VBP'),
 ('suite', 'JJ'),
 ('uses', 'VBZ'),
 ('a', 'DT'),
 ('model-driven', 'JJ'),
 ('abstraction', 'NN'),
 ('layer', 'NN'),
 ('to', 'TO'),
 ('enable', 'VB'),
 ('organizations', 'NNS'),
 ('to', 'TO'),
 ('develop', 'VB'),
 (',', ','),
 ('deploy', 'NN'),
 (',', ','),
 ('and', 'CC'),
 ('operate', 'VBP'),
 ('enterprise', 'NN'),
 ('scale', 'NN'),
 ('ai', 'VBP'),
 ('applications', 'NNS'),
 ('40x', 'CD'),
 ('to', 'TO'),
 ('100x', 'CD'),
 ('faster', 'JJR'),
 ('than', 'IN'),
 ('alternative', 'JJ'),
 ('approaches', 'NNS'),
 ('.', '.'),
 ('www.c3.ai', 'NN'),
 ('as', 'IN'),
 ('a', 'DT'),
 ('data', 'NN'),
 ('scientist', 'NN'),
 (',', ','),
 ('you', 'PRP'

# Extraction of identified POS patterns

In [4]:
# noun phrase
# first pattern for extraction
# an optional determinate followed by any number of adjectives and a noun, plural noun or proper noun
grammar1 = ('''Noun Phrases: {<DT>?<JJ>*<NN|NNS|NNP>+}''')
chunkParser = nltk.RegexpParser(grammar1)
tree1 = chunkParser.parse(tagged)

# typical noun phrase pattern appending to be concatted later
g1_chunks = []
for subtree in tree1.subtrees(filter=lambda t: t.label() == 'Noun Phrases'):
    # print(subtree)
    g1_chunks.append(subtree)

In [5]:
# Noun phrase variation; 
# second pattern for extraction
# preposition maybe, any number of adjective or nouns, any plural nouns or singular nouns
grammar2 = ('''NP2: {<IN>?<JJ|NN>*<NNS|NN>} ''')
chunkParser = nltk.RegexpParser(grammar2)
tree2 = chunkParser.parse(tagged)

# variation of a noun phrase pattern to be pickled for later analyses
g2_chunks = []
for subtree in tree2.subtrees(filter=lambda t: t.label() == 'NP2'):
    # print(subtree)
    g2_chunks.append(subtree)
    

In [6]:
# third pattern for extraction
# any sort of verb followed by any number of nouns
grammar3 = ('''
    VS: {<VBG|VBZ|VBP|VBD|VB|VBN><NNS|NN>*}
    ''')
chunkParser = nltk.RegexpParser(grammar3)
tree3 = chunkParser.parse(tagged)

# verb-noun pattern appending to be concatted later
g3_chunks = []
for subtree in tree3.subtrees(filter=lambda t: t.label() == 'VS'):
    # print(subtree)
    g3_chunks.append(subtree)

In [7]:
# fourth pattern for extraction
# any number of a singular or plural noun followed by a comma followed by the same noun, noun, noun pattern
grammar4 = ('''
    Commas: {<NN|NNS>*<,><NN|NNS>*<,><NN|NNS>*} 
    ''')
chunkParser = nltk.RegexpParser(grammar4)
tree4 = chunkParser.parse(tagged)

# common pattern of listing skills appending to be concatted later
g4_chunks = []
for subtree in tree4.subtrees(filter=lambda t: t.label() == 'Commas'):
    # print(subtree)
    g4_chunks.append(subtree)
    

In [8]:
# sample size reused from prior cell use just in case a sample would be 
# used to reduce the amount of extractions to go through
print('Length:', len(g1_chunks), 'Sample Size:', len(g1_chunks) * .10)
print('Length:', len(g2_chunks), 'Sample Size:', len(g2_chunks) * .10) 
print('Length:', len(g3_chunks), 'Sample Size:', len(g3_chunks) * .10)
print('Length:', len(g4_chunks), 'Sample Size:', len(g4_chunks) * .10)

Length: 119 Sample Size: 11.9
Length: 122 Sample Size: 12.200000000000001
Length: 67 Sample Size: 6.7
Length: 5 Sample Size: 0.5


# Chunk Dataframe

In [9]:
def data_process(chunks):
    '''creates a dataframe that easily parses the chunks data 
    '''
    df = pd.DataFrame(chunks)    
    df.fillna('X', inplace = True)
    
    pattern = []
    for row in df.values:
        phrase = ''
        for tup in row:
            # needs a space at the end for seperation
            phrase += tup[0] + ' '
        phrase = ''.join(phrase)
        # could use padding tages but encoder method will provide during 
        # tokenizing/embeddings; X can replace paddding for now
        pattern.append( phrase.replace('X', '').strip())
    df['phrase'] = pattern
    # only returns 10% of each dataframe to be used 
    return df.phrase

In [10]:
# one training corpus with 10% of each POS regex identification
chunked = pd.concat([data_process(g1_chunks),
                      data_process(g2_chunks), 
                      data_process(g3_chunks),
                      data_process(g4_chunks)], 
                        ignore_index = True )

In [11]:
class PaddingInputExample(object):
        """Fake example so the num input examples is a multiple of the batch size.
      When running eval/predict on the TPU, we need to pad the number of examples
      to be a multiple of the batch size, because the TPU requires a fixed batch
      size. The alternative is to drop the last batch, which is bad because it means
      the entire output data won't be generated.
      We use this class instead of `None` because treating `None` as padding
      battches could cause silent errors.
      """

class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
    Args:
      guid: Unique id for the example.
      text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
      text_b: (Optional) string. The untokenized text of the second sequence.
        Only must be specified for sequence pair tasks.
      label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    bert_module =  hub.Module(bert_path)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    vocab_file, do_lower_case = sess.run(
        [
            tokenization_info["vocab_file"],
            tokenization_info["do_lower_case"],
        ]
    )

    return FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

def convert_single_example(tokenizer, example, max_seq_length=256):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    if isinstance(example, PaddingInputExample):
        input_ids = [0] * max_seq_length
        input_mask = [0] * max_seq_length
        segment_ids = [0] * max_seq_length
        label = 0
        return input_ids, input_mask, segment_ids, label

    tokens_a = tokenizer.tokenize(example.text_a)
    if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0 : (max_seq_length - 2)]

    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    return input_ids, input_mask, segment_ids, example.label

def convert_examples_to_features(tokenizer, examples, max_seq_length=256):
    """Convert a set of `InputExample`s to a list of `InputFeatures`."""

    input_ids, input_masks, segment_ids, labels = [], [], [], []
    for example in tqdm_notebook(examples, desc="Converting examples to features"):
        input_id, input_mask, segment_id, label = convert_single_example(
            tokenizer, example, max_seq_length
        )
        input_ids.append(input_id)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
        labels.append(label)
    return (
        np.array(input_ids),
        np.array(input_masks),
        np.array(segment_ids),
        np.array(labels).reshape(-1, 1),
    )

def convert_text_to_examples(texts, labels):
    """Create InputExamples"""
    InputExamples = []
    for text, label in zip(texts, labels):
        InputExamples.append(
            InputExample(guid=None, text_a=" ".join(text), text_b=None, label=label)
        )
    return InputExamples

In [12]:
# Params for bert model and tokenization
bert_path = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

# function for intializing the variables which should not be forgotten!!!!!!!
def initialize_vars(sess):
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    K.set_session(sess)

In [13]:
unseen = np.array(chunked)
unseen_text = unseen[:, np.newaxis]

# Instantiate tokenizer
tokenizer = create_tokenizer_from_hub_module()

# Convert data to InputExample format
unseen_examples = convert_text_to_examples(unseen_text, np.zeros((len(unseen_text),1)))

# Convert to features
(unseen_input_ids, unseen_input_masks, unseen_segment_ids, unseen_labels 
) = convert_examples_to_features(tokenizer,
                                 unseen_examples,
                                 max_seq_length = 18)



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore



C:\Users\marti\Anaconda3\envs\nlp_env\lib\site-packages\ipykernel_launcher.py:89: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [14]:
unseen_input_ids

array([[  101,  1039,  2509, ...,     0,     0,     0],
       [  101,  6960,  9932, ...,     0,     0,     0],
       [  101,  3617,  8651, ...,     0,     0,     0],
       ...,
       [  101,  3330,  1010, ...,     0,     0,     0],
       [  101, 11208,  1010, ...,     0,     0,     0],
       [  101, 21025,  2705, ...,     0,     0,     0]])

In [15]:
class BertLayer(tf.keras.layers.Layer):
    ''' a function had to be added at the end in order to save the model due to 
    not identifying the shape early in the model build; as a result a function 
    that overrides original configuration file was made from issues section on guthub'''
    
    def __init__(
        self,
        n_fine_tune_layers=10,
        pooling="first",
        bert_path="https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1",
        **kwargs,
    ):
        self.n_fine_tune_layers = n_fine_tune_layers
        self.trainable = True
        self.output_size = 768
        self.pooling = pooling
        self.bert_path = bert_path
        if self.pooling not in ["first", "mean"]:
            raise NameError(
                f"Undefined pooling type (must be either first or mean, but is {self.pooling}"
            )

        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.bert = hub.Module(
            self.bert_path, trainable=self.trainable, name=f"{self.name}_module"
        )

        # Remove unused layers
        trainable_vars = self.bert.variables
        if self.pooling == "first":
            trainable_vars = [var for var in trainable_vars if not "/cls/" in var.name]
            trainable_layers = ["pooler/dense"]

        elif self.pooling == "mean":
            trainable_vars = [
                var
                for var in trainable_vars
                if not "/cls/" in var.name and not "/pooler/" in var.name
            ]
            trainable_layers = []
        else:
            raise NameError(
                f"Undefined pooling type (must be either first or mean, but is {self.pooling}"
            )

        # Select how many layers to fine tune
        for i in range(self.n_fine_tune_layers):
            trainable_layers.append(f"encoder/layer_{str(11 - i)}")

        # Update trainable vars to contain only the specified layers
        trainable_vars = [
            var
            for var in trainable_vars
            if any([l in var.name for l in trainable_layers])
        ]

        # Add to trainable weights
        for var in trainable_vars:
            self._trainable_weights.append(var)

        for var in self.bert.variables:
            if var not in self._trainable_weights:
                self._non_trainable_weights.append(var)

        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        input_ids, input_mask, segment_ids = inputs
        bert_inputs = dict(
            input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids
        )
        if self.pooling == "first":
            pooled = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)[
                "pooled_output"
            ]
        elif self.pooling == "mean":
            result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)[
                "sequence_output"
            ]

            mul_mask = lambda x, m: x * tf.expand_dims(m, axis=-1)
            masked_reduce_mean = lambda x, m: tf.reduce_sum(mul_mask(x, m), axis=1) / (
                    tf.reduce_sum(m, axis=1, keepdims=True) + 1e-10)
            input_mask = tf.cast(input_mask, tf.float32)
            pooled = masked_reduce_mean(result, input_mask)
        else:
            raise NameError(f"Undefined pooling type (must be either first or mean, but is {self.pooling}")

        return pooled

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_size)

    def get_config(self):
        config = super(BertLayer, self).get_config().copy()
        config.update({
            'n_fine_tune_layers': self.n_fine_tune_layers,
            # 'trainable': self.trainable,
            # 'output_size': self.output_size,
            'pooling': self.pooling,
            'bert_path': self.bert_path,
        })

        return config

# Network Archetecture

In [16]:
# Build model lines indicated where to change the model when tweaking
def build_model(max_seq_length = 18): 
    # three inputs for id, mask and segments of each phrase
    in_id = tf.keras.layers.Input(shape=(max_seq_length,), name="input_ids")
    in_mask = tf.keras.layers.Input(shape=(max_seq_length,), name="input_masks")
    in_segment = tf.keras.layers.Input(shape=(max_seq_length,), name="segment_ids")
    bert_inputs = [in_id, in_mask, in_segment]
    # first layer where bert embeddings are given to the "bert-accepted"tokenized inputs
    bert_output = BertLayer(n_fine_tune_layers=10)(bert_inputs)
    # five dense layers, decreasing width with each layer
    x = tf.keras.layers.Dense(128, activation='relu')(bert_output)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.Dense(24, activation='tanh')(x)
    x = tf.keras.layers.Dense(12, activation='relu')(x)
    # softmax activation on final layer to deliver probability values of predictions
    pred = tf.keras.layers.Dense(1, activation='softmax')(x)
    # optimizer remained throughout iterations
    optimizer = tf.keras.optimizers.Adam(lr=0.00001)
    # load em up
    model = tf.keras.models.Model(inputs=bert_inputs, outputs=pred)
    # compied and ready to run; metrics stayed the same accuracy added for real time analysis of training
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

In [18]:
# # # Clear and load model existing from prior use of cell in colab notebook
model = None
model = build_model(18)
#initializing the variables which can often be forgotten
initialize_vars(sess)
# C:\Users\marti\Job Description Analysis\POS Tagging & Classification Model
model.load_weights(r'C:\Users\marti\Job Description Analysis\POS Tagging & Classification Model\extraction_model_weights.h5')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 18)]         0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 18)]         0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 18)]         0                                            
__________________________________________________________________________________________________
bert_layer_1 (BertLayer)        (None, 768)          110104890   input_ids[0][0]                  
                                                                 input_masks[0][0]          

In [19]:
# # predictions after we clear and reload model
save_preds = model.predict([unseen_input_ids, 
                            unseen_input_masks, 
                            unseen_segment_ids]
                              ) 

In [20]:
predictions = pd.DataFrame(dict(list(zip(unseen, save_preds))))
predictions = predictions.T
# pd.set_option('display.max_rows', None)
# predictions.to_csv('predictions2.csv')

In [27]:
pd.options.display.max_rows = None
predictions

,0
c3.ai,1.0
enterprise ai software provider,1.0
digital transformation,1.0
proven c3,1.0
a model-driven abstraction layer,1.0
organizations,1.0
deploy,1.0
enterprise scale,1.0
applications,1.0
alternative approaches,1.0
